In [1]:
import pandas as pd
import math
import os

In [125]:
class nsf_wqi:

    
    def __init__(self, data_path = 'calculate' ):
        self.q_data = pd.read_csv(os.path.join(data_path, 'Q_calculate.csv'))
        self.temp_solubility_data = pd.read_csv(os.path.join(data_path, 'temp_solubility_calculate.csv')).dropna()
        self.weight_data = pd.read_csv(os.path.join(data_path, 'weight.csv'))
        
    def calc_qi(self,item, input_value):
        #try:
            q_data_item = self.q_data[[item, item+'QI']].dropna()
            water_item = dict(q_data_item[item])
            water_item_qi = dict(q_data_item[item+'QI'])
            
            input_value = input_value
            first = False
            last = False
            
            for idx, val in water_item.items():
                if input_value <= water_item[0]:
                    find_idx = 0
                    first = True
                    break
                if idx == len(water_item)-1:
                   find_idx = idx
                   last = True
                   continue
               
                if input_value >= val and input_value <water_item[idx+1]:
                    find_idx = idx
                    break
                
            if first:
                qi_value = water_item_qi[0]
            elif last:
                qi_value = water_item_qi[len(water_item)-1]
            else:
                w1 = abs(input_value - water_item[find_idx+1])/(water_item[find_idx+1]-water_item[find_idx])
                w2 = abs(input_value - water_item[find_idx])/(water_item[find_idx+1]-water_item[find_idx])
                qi_value = w1*water_item_qi[find_idx] + w2*water_item_qi[find_idx+1]
            
            return qi_value
        #except:
        #    print('No item in data item must in {}'.format(str(['BOD','Ecoli','Nitrate','pH','TempChange','TDS','Phosphate','Turbidity','DO_percent'])))
            
    def calc_solubility(self, temp, DO):
        solubility_data = dict(self.temp_solubility_data)
        temp = round(temp)
        solubility = solubility_data['Solubility'][temp]
        DO_percent = DO/solubility
        return DO_percent*100
        
    def calc_weight(self, item_list):
        weight_data = dict(self.weight_data)
        value_sum = 0
        weight_dict = {}
        
        
        for item, value in zip(weight_data['item'],weight_data['value']):
            if item in item_list:
                value_sum += value
                weight_dict[item] = value
        
        for item in weight_dict:
            weight_dict[item] = round(weight_dict[item]/value_sum, 4)
        
        return weight_dict
    
    def calc_wqi(self, wqs_result, print_detail = False):
        
        nsf_wqi_result = {'q':{}, 'w':{}}
        items = list(wqs_result.keys())
        if 'DO' in items and 'Temp' in items:
            DO_percent = self.calc_solubility(wqs_result['Temp'],wqs_result['DO'])    
            del wqs_result['DO']
            del wqs_result['Temp']
            wqs_result['DO_percent'] = DO_percent
        
        items = list(wqs_result.keys())
        weight = self.calc_weight(items)
        nsfw_score = 0
        
        for item in weight.keys():
            nsf_wqi_result['w'][item] = weight[item] 
            nsf_wqi_result['q'][item] = self.calc_qi(item, wqs_result[item])
            #print(item, weight, nsf_wqi_result['w'][item],nsf_wqi_result['q'][item])
            nsfw_score += round(nsf_wqi_result['w'][item]*nsf_wqi_result['q'][item],2)
        
        if print_detail:
            print(nsf_wqi_result)
            print(nsfw_score)
        
        return nsfw_score 

40.0

In [117]:
nsf_wqi().calc_qi('DO_percent', 110)

96.0

In [118]:
print(nsf_wqi().calc_wqi( {'BOD': 1.74, 'pH': 7.98, 'TDS': 7.6, 'Nitrate': 2.63, 'Phosphate': 1.32, 'DO': 12.64, 'Temp':5.6},print_detail=True))

{'q': {'DO_percent': 99.0, 'BOD': 85.2, 'pH': 84.6, 'TDS': 81.28, 'Nitrate': 91.85, 'Phosphate': 33.733333333333334}, 'w': {'DO_percent': 0.2576, 'BOD': 0.1667, 'pH': 0.1667, 'TDS': 0.1061, 'Nitrate': 0.1515, 'Phosphate': 0.1515}}
82
82


In [47]:
test = pd.read_csv('test.csv')

In [48]:
test_dict = dict(test)

In [119]:
datas = {}
idx = 0
for site in test_dict['Site']:
    datas[site] = {}
    for item in test_dict.keys():
        if item == 'Site':
            continue
        if pd.isna(test_dict[item][idx]):
            continue
            
        datas[site][item] = test_dict[item][idx]
    idx +=1

In [126]:
for i in datas:
    print(nsf_wqi().calc_wqi( datas[i]))

81.45
83.48
91.02
77.60999999999999
82.32000000000001
84.06
87.47
85.11999999999999
84.06
74.74999999999999


In [91]:
nsf_wqi().calc_wqi({'BOD': 1.74, 'pH': 7.98, 'TDS': 7.6, 'Nitrate': 2.63, 'Phosphate': 1.32, 'DO': 12.64, 'Temp':5.6}, print_detail=True)

{'q': {'DO_percent': 22, 'BOD': 85.2, 'pH': 20.0, 'TDS': 81.28, 'Nitrate': 91.85, 'Phosphate': 33.733333333333334}, 'w': {'DO_percent': 0.2576, 'BOD': 0.1667, 'pH': 0.1667, 'TDS': 0.1061, 'Nitrate': 0.1515, 'Phosphate': 0.1515}}
51


51

In [63]:
print(nsf_wqi().calc_wqi( datas['k10']))

DO_percent 23


ValueError: cannot convert float NaN to integer

In [53]:
datas

{'k1': {'BOD': 1.74,
  'pH': 7.98,
  'TDS': 7.6,
  'Nitrate': 2.63,
  'Phosphate': 1.32,
  'DO_percent': 101.12},
 'k2': {'BOD': 4.1,
  'pH': 8.53,
  'TDS': 8.8,
  'Nitrate': 1.32,
  'Phosphate': 0.076,
  'DO_percent': 91.14583333333334},
 'k3': {'BOD': 1.82,
  'pH': 7.94,
  'TDS': 13.0,
  'Nitrate': 2.85,
  'Phosphate': 0.063,
  'DO_percent': 98.75},
 'k4': {'BOD': 4.68,
  'pH': 9.3,
  'TDS': 12.8,
  'Nitrate': 1.69,
  'Phosphate': 0.108,
  'DO_percent': 115.85106382978722},
 'k5': {'BOD': 2.98,
  'pH': 7.72,
  'TDS': 10.0,
  'Nitrate': 5.3,
  'Phosphate': 0.295,
  'DO_percent': 94.32432432432434},
 'k6': {'BOD': 4.12,
  'pH': 7.73,
  'TDS': 6.4,
  'Nitrate': 3.12,
  'Phosphate': 0.152,
  'DO_percent': 81.95652173913044},
 'k7': {'BOD': 1.2,
  'pH': 8.02,
  'TDS': 19.8,
  'Nitrate': 4.38,
  'Phosphate': 0.085,
  'DO_percent': 88.85496183206108},
 'k8': {'BOD': 1.82,
  'pH': 7.75,
  'TDS': 12.0,
  'Nitrate': 1.6,
  'Phosphate': 0.063,
  'DO_percent': 67.23404255319149},
 'k9': {'BOD': 

In [22]:
datas

{'k1': {'BOD': 1.74,
  'pH': 7.98,
  'TDS': 7.6,
  'Turb': nan,
  'Nitrate': 2.63,
  'Ecoli': nan,
  'Phosphate': 1.32,
  'DO_percent': 101.12},
 'k2': {'Temp': 18.3,
  'DO': 8.75,
  'BOD': 4.1,
  'pH': 8.53,
  'TDS': 8.8,
  'Turb': nan,
  'Nitrate': 1.32,
  'Ecoli': nan,
  'Phosphate': 0.076},
 'k3': {'Temp': 5.2,
  'DO': 12.64,
  'BOD': 1.82,
  'pH': 7.94,
  'TDS': 13.0,
  'Turb': nan,
  'Nitrate': 2.85,
  'Ecoli': nan,
  'Phosphate': 0.063},
 'k4': {'Temp': 19.2,
  'DO': 10.89,
  'BOD': 4.68,
  'pH': 9.3,
  'TDS': 12.8,
  'Turb': nan,
  'Nitrate': 1.69,
  'Ecoli': nan,
  'Phosphate': 0.108},
 'k5': {'Temp': 11.1,
  'DO': 10.47,
  'BOD': 2.98,
  'pH': 7.72,
  'TDS': 10.0,
  'Turb': nan,
  'Nitrate': 5.3,
  'Ecoli': nan,
  'Phosphate': 0.295},
 'k6': {'Temp': 19.7,
  'DO': 7.54,
  'BOD': 4.12,
  'pH': 7.73,
  'TDS': 6.4,
  'Turb': nan,
  'Nitrate': 3.12,
  'Ecoli': nan,
  'Phosphate': 0.152},
 'k7': {'Temp': 4.3,
  'DO': 11.64,
  'BOD': 1.2,
  'pH': 8.02,
  'TDS': 19.8,
  'Turb': nan,

In [14]:
a = nsf_wqi()
a.q_data

,BOD,BODQI,Ecoli,EcoliQI,Nitrate,NitrateQI,pH,pHQI,TempChange,TempChangeQI,TDS,TDSQI,Phosphate,PhosphateQI,Turbidity,TurbidityQI,DO_percent,DO_percentQI
0,0.00000,100.0,1.0,100.0,0.00000,97.0,0.00000,1.0,-10.00000,55.0,0.00000,79.0,0.00000,100.0,0.00000,99.0,0.00000,2
1,0.50000,98.0,2.0,90.0,2.00000,95.0,2.00000,2.0,0.00000,93.0,10.00000,82.0,0.20000,92.0,3.00000,90.0,5.00000,5
2,1.00000,95.0,5.0,81.0,3.00000,90.0,3.00000,4.0,5.00000,73.0,20.00000,84.0,0.50000,60.0,8.00000,80.0,10.00000,7
3,1.50000,90.0,10.0,72.0,3.50000,80.0,3.50000,6.0,9.50000,47.0,30.00000,84.5,0.70000,50.0,13.00000,70.0,16.00000,10
4,2.50000,70.0,20.0,65.0,4.00000,70.0,4.00000,9.0,12.00000,36.0,40.00000,86.0,1.00000,40.0,15.00000,67.0,20.00000,12
5,4.00000,61.0,50.0,56.0,6.00000,60.0,4.10000,10.0,14.50000,32.0,60.00000,87.0,1.30000,34.0,20.00000,61.0,25.00000,17
6,6.50000,48.0,100.0,47.0,10.00000,51.0,4.50000,15.0,21.00000,20.0,70.00000,86.0,1.60000,30.0,30.00000,53.0,32.00000,20
7,8.50000,40.0,200.0,38.0,17.00000,40.0,4.80000,20.0,30.00001,10.0,150.00000,79.0,2.00000,27.0,40.00000,45.0,34.00000,22
8,11.00000,30.0,500.0,28.0,27.00000,30.0,5.10000,30.0,NaN,NaN,450.00000,40.0,3.20000,20.0,50.00000,39.0,40.00000,30
9,15.00000,20.0,1000.0,26.0,37.00000,20.0,6.20000,60.0,NaN,NaN,500.00001,31.0,4.00000,17.0,60.00000,33.0,45.00000,37
